In [1]:
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import numpy as np
import os
import cv2
import time
import pandas as pd

In [2]:
def load_images():
    dir = os.path.join(os.sep,'home','mohit','Desktop','dataset')
    images = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.imread(os.path.join(dir,'Cheque'+str(k),i,i+'_'+j+'.tif'))
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.cvtColor(images['c'+str(k)+'_'+i+'_'+j], cv2.COLOR_BGR2YCrCb)
                except:
                    break
    return images

In [3]:
def n_size(images):
    size = {} #no. of ink pixels .. considering all non white pixels(<th) as ink pixels
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                count = 0
                try:
                    image = images['c'+str(k)+'_'+i+'_'+j][:,:,0]
                    for a in range(0,image.shape[0]):
                        for b in range(0,image.shape[1]):
                            if image[a][b] != 255:
                                count+=1
                    size['c'+str(k)+'_'+i+'_'+j] = count
                   # print('c'+str(k)+'_'+i+'_'+j,count)
                except:
                    #print('..')
                    break  
    return size

In [4]:
def mask(image):
    r = (image==255)
    return r


In [26]:
def get_pixel_else_0(l, idx, idy, default=0):
    try:
        return l[idx,idy]
    except IndexError:
        return default

def thresholded(center, pixels):
    out = []
    for a in pixels:
        if a >= center:
            out.append(1)
        else:
            out.append(0)
    return out

def s(x1,x2):
    if x2>=x1:
        return 1
    else:
        return 0
    
def find_variations(pixel_values):
    prev = pixel_values[-1]
    t = 0
    for p in range(0, len(pixel_values)):
        cur = pixel_values[p]
        if cur != prev:
            t += 1
        prev = cur
    return t    

def ELBP_CI(image,beta):
    return (image>beta)

def calc_beta(img,img1,size):
    beta = 0
    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            if(img1[x,y]!=255):        
                beta+=img[x,y]
    beta/=size
    return beta

def ELBP_NI(image,encoding='riu2'):
    result = np.zeros_like(image,dtype=np.float64)
    for x in range(0, len(image)):
        for y in range(0, len(image[0])):
            b = 0
            
            top_left      = get_pixel_else_0(image, x-1, y-1)
            top_up        = get_pixel_else_0(image, x, y-1)
            top_right     = get_pixel_else_0(image, x+1, y-1)
            right         = get_pixel_else_0(image, x+1, y )
            left          = get_pixel_else_0(image, x-1, y )
            bottom_left   = get_pixel_else_0(image, x-1, y+1)
            bottom_right  = get_pixel_else_0(image, x+1, y+1)
            bottom_down   = get_pixel_else_0(image, x,   y+1 )
            
            b=np.mean([top_left, top_up, top_right, right, bottom_right,bottom_down, bottom_left, left])

            values = thresholded(b, [top_left, top_up, top_right, right, bottom_right,bottom_down, bottom_left, left])
            
            if(encoding=='riu2'):
                if(find_variations(values)<=2):
                    result.itemset((x,y), values.count(0))
                else :
                    result.itemset((x,y),8)
            else:
                weights = [1, 2, 4, 8,16,32,64,128]
                res = 0
                for a in range(0, len(weights)):
                    res += weights[a] * values[a]
                result.itemset((x,y),res)
            
    return result


def ELBP_RD(image,encoding='riu2'):
    r= np.zeros_like(image,dtype=np.float64)

    for x in range(0, len(image)):
        for y in range(0, len(image[0])):
            values = []
            top_left1,top_left2          = get_pixel_else_0(image, x-1, y-1),get_pixel_else_0(image, x-2, y-2)
            top_up1,top_up2              = get_pixel_else_0(image, x, y-1),get_pixel_else_0(image, x, y-2)
            top_right1,top_right2        = get_pixel_else_0(image, x+1, y-1),get_pixel_else_0(image, x+2, y-2)
            right1,right2                = get_pixel_else_0(image, x+1, y ),get_pixel_else_0(image, x+2, y )
            left1,left2                  = get_pixel_else_0(image, x-1, y ),get_pixel_else_0(image, x-2, y )
            bottom_left1,bottom_left2    = get_pixel_else_0(image, x-1, y+1),get_pixel_else_0(image, x-2, y+2)
            bottom_right1,bottom_right2  = get_pixel_else_0(image, x+1, y+1),get_pixel_else_0(image, x+2, y+2)
            bottom_down1,bottom_down2    = get_pixel_else_0(image, x,   y+1 ),get_pixel_else_0(image, x,   y+2 )
            
            values.append(s(top_left1,top_left2))
            values.append(s(top_up1,top_up2))
            values.append(s(top_right1,top_right2))
            values.append(s(right1,right2))
            values.append(s(bottom_right1,bottom_right2))
            values.append(s(bottom_down1,bottom_down2))
            values.append(s(bottom_left1,bottom_left2))
            values.append(s(left1,left2))
            
            if(encoding=='riu2'):
                if(find_variations(values)<=2):
                    r.itemset((x,y), values.count(0))
                else :
                    r.itemset((x,y),9)
            else:
                weights = [1, 2, 4, 8,16,32,64,128]
                res = 0
                for a in range(0, len(weights)):
                    res += weights[a] * values[a]
                r.itemset((x,y),res)
                
    return r

In [28]:
def Extended_LBP(images,dim,sizes,encoding='riu2'):
    hists_ni = {}
    hists_rd = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    print(k,i,j)
                    s = sizes['c'+str(k)+'_'+i+'_'+j]

                    ni_pens = ELBP_NI(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],encoding)
                    rd_pens = ELBP_RD(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],encoding)

                    beta = calc_beta(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],images['c'+str(k)+'_'+i+'_'+j][:,:,0],s)
                    masc = ELBP_CI(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],beta)

                    if(encoding=='riu2'):
                        ni_pens+=9*masc
                        rd_pens+=10*masc
                        r1=19
                        r2=21
                    else:
                        ni_pens+=256*masc
                        rd_pens+=256*masc
                        r1=513
                        r2=513

                    rd_pens+= (mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*1000)
                    ni_pens+= (mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*1000)


                    hist_ni,_ = np.histogram(ni_pens.ravel(),range(0,r1))
                    hist_rd,_ = np.histogram(rd_pens.ravel(),range(0,r2))

                    hists_ni['c'+str(k)+'_'+i+'_'+j] = hist_ni/s
                    hists_rd['c'+str(k)+'_'+i+'_'+j] = hist_rd/s

                except:
                    break
                   
    return hists_ni,hists_rd

In [20]:
def compute_dist(param1,param2,n1,n2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def distance_distribution(features,sizes):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' same {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' diff {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                        except:
                            
                            pass
     
    print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

In [8]:
images = load_images()
sizes = n_size(images)

# using riu2 encoding

In [9]:
hist_Y_ni,hist_Y_rd = Extended_LBP(images,0,sizes)
hist_Cb_ni,hist_Cb_rd = Extended_LBP(images,1,sizes)
hist_Cr_ni,hist_Cr_rd = Extended_LBP(images,2,sizes)

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1
16 2 2
16 2 3
17 1 1
17 1 2
17 1 3
17 1 4
17 1 5
17 1 6
17 2 1
17 2 2
17 2 3
18 1 1
18 1 2
18 1 3
18 1 4
18 1 5
18 1 6
18 1 7
18 2 1
18 2 2
18 2 3
18 2 4
19 1 1
19 1 2
19 1 3
19 1 4
19 1 5
19 1 6
19 2 1
19 2 2
19 2 3
20 1 1
20 1 2
20 1 3
20 1 4
20 1 5
20 1 6
20 1 7
20 2 1
20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1

20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1
26 1 2
26 1 3
26 1 4
26 1 5
26 1 6
26 2 1
26 2 2
26 2 3
26 2 4
27 1 1
27 1 2
27 1 3
27 1 4
27 1 5
27 2 1
27 2 2
27 2 3
27 2 4
27 2 5
28 1 1
28 1 2
28 1 3
28 1 4
28 1 5
28 2 1
28 2 2
28 2 3
28 2 4
29 1 1
29 1 2
29 1 3
29 1 4
29 1 5
29 1 6
29 1 7
29 2 1
29 2 2
29 2 3
30 1 1
30 1 2
30 1 3
30 1 4
30 1 5
30 2 1
30 2 2
30 2 3
30 2 4
31 1 1
31 1 2
31 1 3
31 1 4
31 1 5
31 1 6
31 1 7
31 2 1
31 2 2
31 2 3
32 1 1
32 1 2
32 1 3
32 1 4
32 1 5
32 1 6
32 1 7
32 2 1
32 2 2
32 2 3
32 2 4
33 1 1
33 1 2
33 1 3
33 1 4
33 1 5
33 1 6
33 2 1
33 2 2
33 2 3
33 2 4
34 1 1
34 1 2
34 1 3
34 1 4
34 1 5
34 1 6
34 1 7
34 2 1
34 2 2
34 2 3
35 1 1
35 1 2
35 1 3

In [10]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y_ni,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb_ni,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr_ni,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
31.104587587273784 31.308878296232898 24.259897017209106


In [11]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y_rd,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb_rd,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr_rd,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
24.308442715814216 30.862749583771173 21.249678336947426


In [12]:

Y = pd.DataFrame(hist_Y_ni)
Cb = pd.DataFrame(hist_Cb_ni)
Cr = pd.DataFrame(hist_Cr_ni)

all_features_ni = pd.concat([Y,Cb,Cr])
all_features_ni.to_csv('/home/mohit/Desktop/featuresUpdated/ELBP_ni|ci_riu2.csv',sep = ',')



In [34]:
Y = pd.DataFrame(hist_Y_rd)
Cb = pd.DataFrame(hist_Cb_rd)
Cr = pd.DataFrame(hist_Cr_rd)

all_features_rd = pd.concat([Y,Cb,Cr])
all_features_rd.to_csv('/home/mohit/Desktop/featuresUpdated/ELBP_rd|ci_riu2.csv',sep = ',')

In [15]:
all_features_ni

,c100_1_1,c100_1_2,c100_1_3,c100_1_4,c100_2_1,c100_2_2,c100_2_3,c101_1_1,c101_1_2,c101_1_3,...,c99_2_3,c9_1_1,c9_1_2,c9_1_3,c9_1_4,c9_1_5,c9_1_6,c9_1_7,c9_2_1,c9_2_2
0,0.000000,0.000000,0.000000,0.000000,0.000354,0.000000,0.000000,0.000319,0.000000,0.000562,...,0.000000,0.000000,0.000000,0.000000,0.000575,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000387,0.000000,0.000000,0.000304,0.000000,0.000000,0.000533,0.000638,0.002341,0.000562,...,0.000737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.003484,0.004021,0.002126,0.004562,0.001416,0.001546,0.002131,0.000638,0.001171,0.003371,...,0.000000,0.000000,0.000665,0.000000,0.000575,0.000596,0.002343,0.000000,0.000000,0.000803
3,0.020906,0.024698,0.020553,0.023723,0.012389,0.010823,0.012786,0.019127,0.014436,0.015169,...,0.013274,0.008322,0.013298,0.010595,0.023577,0.014009,0.019918,0.019730,0.007212,0.015261
4,0.039876,0.040207,0.048193,0.045925,0.050265,0.042134,0.059670,0.072362,0.066329,0.061798,...,0.035398,0.041609,0.051197,0.076610,0.089132,0.098063,0.080844,0.084112,0.073317,0.064659
5,0.134727,0.143021,0.170801,0.163625,0.248142,0.270584,0.240277,0.240038,0.203277,0.232022,...,0.295723,0.162275,0.204787,0.195599,0.202990,0.197019,0.216169,0.204915,0.259615,0.292369
6,0.102207,0.085009,0.078668,0.081509,0.099823,0.100503,0.104422,0.103921,0.114709,0.100562,...,0.101032,0.110957,0.101729,0.070090,0.047729,0.056334,0.064441,0.066113,0.066106,0.059438
7,0.024390,0.020103,0.007796,0.016119,0.010973,0.010050,0.019180,0.023589,0.025361,0.017978,...,0.017699,0.029126,0.009973,0.004890,0.002875,0.005067,0.002343,0.006923,0.003606,0.003213
8,0.223384,0.211373,0.194897,0.204684,0.183363,0.169308,0.171550,0.142812,0.177136,0.173034,...,0.152655,0.156727,0.139628,0.127954,0.126509,0.116841,0.123609,0.118726,0.127404,0.115261
9,0.000000,0.000000,0.000000,0.000000,0.001062,0.000000,0.001066,0.000000,0.001951,0.000562,...,0.000000,0.001387,0.000000,0.000000,0.000000,0.000596,0.000000,0.000000,0.000000,0.000000


# using default encoding

In [29]:
hist_Y_ni,hist_Y_rd = Extended_LBP(images,0,sizes,encoding='default')
hist_Cb_ni,hist_Cb_rd = Extended_LBP(images,1,sizes,encoding='default')
hist_Cr_ni,hist_Cr_rd = Extended_LBP(images,2,sizes,encoding='default')

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1
16 2 2
16 2 3
17 1 1
17 1 2
17 1 3
17 1 4
17 1 5
17 1 6
17 2 1
17 2 2
17 2 3
18 1 1
18 1 2
18 1 3
18 1 4
18 1 5
18 1 6
18 1 7
18 2 1
18 2 2
18 2 3
18 2 4
19 1 1
19 1 2
19 1 3
19 1 4
19 1 5
19 1 6
19 2 1
19 2 2
19 2 3
20 1 1
20 1 2
20 1 3
20 1 4
20 1 5
20 1 6
20 1 7
20 2 1
20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1

20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1
26 1 2
26 1 3
26 1 4
26 1 5
26 1 6
26 2 1
26 2 2
26 2 3
26 2 4
27 1 1
27 1 2
27 1 3
27 1 4
27 1 5
27 2 1
27 2 2
27 2 3
27 2 4
27 2 5
28 1 1
28 1 2
28 1 3
28 1 4
28 1 5
28 2 1
28 2 2
28 2 3
28 2 4
29 1 1
29 1 2
29 1 3
29 1 4
29 1 5
29 1 6
29 1 7
29 2 1
29 2 2
29 2 3
30 1 1
30 1 2
30 1 3
30 1 4
30 1 5
30 2 1
30 2 2
30 2 3
30 2 4
31 1 1
31 1 2
31 1 3
31 1 4
31 1 5
31 1 6
31 1 7
31 2 1
31 2 2
31 2 3
32 1 1
32 1 2
32 1 3
32 1 4
32 1 5
32 1 6
32 1 7
32 2 1
32 2 2
32 2 3
32 2 4
33 1 1
33 1 2
33 1 3
33 1 4
33 1 5
33 1 6
33 2 1
33 2 2
33 2 3
33 2 4
34 1 1
34 1 2
34 1 3
34 1 4
34 1 5
34 1 6
34 1 7
34 2 1
34 2 2
34 2 3
35 1 1
35 1 2
35 1 3

In [30]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y_ni,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb_ni,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr_ni,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
23.513892776828925 28.837197794186917 22.67301560547345


In [31]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y_rd,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb_rd,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr_rd,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
24.865086027152692 21.271438072524493 13.876209319729536


In [32]:

Y = pd.DataFrame(hist_Y_ni)
Cb = pd.DataFrame(hist_Cb_ni)
Cr = pd.DataFrame(hist_Cr_ni)

all_features_ni = pd.concat([Y,Cb,Cr])
all_features_ni.to_csv('/home/mohit/Desktop/featuresUpdated/ELBP_ni|ci_default.csv',sep = ',')



In [33]:
Y = pd.DataFrame(hist_Y_rd)
Cb = pd.DataFrame(hist_Cb_rd)
Cr = pd.DataFrame(hist_Cr_rd)

all_features_rd = pd.concat([Y,Cb,Cr])
all_features_rd.to_csv('/home/mohit/Desktop/featuresUpdated/ELBP_rd|ci_default.csv',sep = ',')